In [1]:
import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

In [2]:
config_list = autogen.config_list_openai_aoai()
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、gemini、gemini_vision
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)

In [3]:

llm_config = {
    "seed": 42,  # seed for caching and reproducibility
    "config_list": config_list_gemini,  # a list of GeminiOpenAI API configurations
    "temperature": 0,  # temperature for sampling
    # "use_cache": True,  # whether to use cache
}
# create an AssistantAgent named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config= llm_config,
)
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    llm_config= llm_config,
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)


[autogen.oai.client: 02-24 09:59:18] {124} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 02-24 09:59:19] {124} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [4]:
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyBa***************************AA3w. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [29]:
chat_messages = user_proxy.chat_messages[assistant]

In [31]:
chat_messages[1]

{'content': 'First, let\'s get the current date using Python. \n\n```python\n# python code\nfrom datetime import date\n\n# Get today\'s date\ntoday = date.today()\n\nprint("Today\'s date:", today)\n```\n\nNext, we need to fetch the stock prices for META (Facebook) and TESLA for the current year. We can use the `yfinance` library in Python to fetch this data. If it\'s not installed, you can install it using pip:\n\n```shell\n# shell script\npip install yfinance\n```\n\nAfter installing `yfinance`, we can fetch the stock prices:\n\n```python\n# python code\nimport yfinance as yf\n\n# Get data for the current year\nstart_date = f"{today.year}-01-01"\nend_date = str(today)\n\n# Get data for META (Facebook)\nmeta_data = yf.download(\'FB\', start=start_date, end=end_date)\n\n# Get data for TESLA\ntesla_data = yf.download(\'TSLA\', start=start_date, end=end_date)\n\n# Calculate the year-to-date gain for each stock\nmeta_gain = ((meta_data[\'Close\'][-1] - meta_data[\'Close\'][0]) / meta_data[